# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [174]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [175]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

c:\Downloads\Courses\Udacity_Data_Engineering\Data_Modelling_Cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [176]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


PermissionError: [Errno 13] Permission denied: 'event_datafile_new.csv'

In [177]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [178]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
try:
    session = cluster.connect()
except Exception as e:
    print(e)
# To establish connection and begin executing queries, need a session


#### Create Keyspace

In [179]:
try:
    session.execute("CREATE KEYSPACE if not exists sprakify with REPLICATION ="+
                    "{'class':'SimpleStrategy','replication_factor':1}")
except Exception as e:
    print(e)


#### Set Keyspace

In [180]:
try:
    session.set_keyspace("sprakify")
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## **artist_song_by_session table creation and design:**

It is about getting the artist,song and song length using sessionId and itemInSession 
columns.The table creation will contain the mentioned columns, and the partition 
key will be the sessionId. The purpose for this is it will be used in the where 
condition, and it will cause the rows to be evenly distributed. 
The clustering column is the itemInSesison. Another soution to have the 
parition key to be composite of (SessionId,itemInSession) but the 
performance gain is not that much.

In [182]:
query="CREATE TABLE IF NOT EXISTS artist_song_by_session " \
      "(sessionId INT,iteminSession INT,artist text,song text,length decimal,"\
      "PRIMARY KEY((sessionId,iteminSession)));"
try:
    session.execute(query)
except Exception as e:
    print("Error in creating table artist_song_by_session",e)

                    

## **Loading the table artist_song_by_session:**

It is a loop on each line in the file 'event_datafile_new.csv' to insert the 
required fields in table 'artist_song_by_session'. Some sort of rejection management mechanism has been added to detect in error happens in inserting any row in the table.

In [183]:
file = 'event_datafile_new.csv'
filerejected='rejected_event_datafile_new_q1.csv'
flag = False
with open(file, encoding = 'utf8') as f,open(filerejected,'w',encoding='utf8',newline='') as freject:
    csvreader = csv.reader(f)
    writer = csv.writer(freject, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_session(sessionId,iteminSession,artist,song,length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        try:
            session.execute(query,(int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
        except Exception as e:
            if not flag:
                print(e)
            flag=True
            writer.writerow((line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],line[9],line[10]))

if flag:
    print("Please check the rejected file: ",filerejected)
else:
    os.remove(filerejected)

## **Verification for the artist_song_by_session design:**

The commented code is all about getting row count for loading verification. A select statement which represents the first query has been written and executed for testing and verifying the results.

In [184]:
#rows=session.execute("SELECT count(*) FROM artist_song")
#for row in rows:
#    print(row)
    
try:
    rows=session.execute("SELECT ARTIST,SONG,length FROM artist_song_by_session WHERE sessionId = 338 and itemInSession = 4")
    for row in rows:
        print(f'Artist:{row.artist}\nSong:{row.song}\nLength:{row.length}')
except Exception as e:
    print("Error in executingquery for Q1: ",e)
    

Artist:Faithless
Song:Music Matters (Mark Knight Dub)
Length:495.3073


## **artist_song_by_user table creation and design:**

It is required to get the artist,song and username(first and last)sorted by 
itemInSession given that we have certain userId and sessionId.
The table creation will include these columns in addition to  primary key.
The partition key will be composite key (userId,sessionId) to enhance
the query performance. The clustering key will be itemInSession which is required
to get the results sorted using this column.

In [164]:

query="CREATE TABLE IF NOT EXISTS artist_song_by_user " \
      "(userId INT,sessionId INT,iteminSession INT,artist text,song text,firstName text,lastName text,"\
      "PRIMARY KEY((userId,sessionId),iteminSession));"
try:
    session.execute(query)
except Exception as e:
    print("Error in creating table artist_song_by_user",e)

                    

## **Loading the table artist_song_by_user:**

It is a loop on each line in the file 'event_datafile_new.csv' to insert the 
required fields in table 'artist_song_by_user'. Some sort of rejection management mechanism has been added to detect in error happens in inserting any row in the table.

In [165]:
file = 'event_datafile_new.csv'
filerejected='rejected_event_datafile_new_q2.csv'
flag = False
with open(file, encoding = 'utf8') as f,open(filerejected,'w',encoding='utf8',newline='') as freject:
    csvreader = csv.reader(f)
    writer = csv.writer(freject, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_by_user(userId,sessionId,iteminSession,artist,song,firstName,lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        try:
            session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
        except Exception as e:
            if not flag:
                print(e)
            flag=True
            writer.writerow((line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],line[9],line[10]))

if flag:
    print("Please check the rejected file: ",filerejected)
else:
    os.remove(filerejected)

In [166]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
## I added in the output of the select iteminsession to make sure that the output is sorted by it.
#rows=session.execute("SELECT count(*) FROM artist_song_by_user")
#for row in rows:
#    print(row)
try:
    rows=session.execute("SELECT ARTIST,SONG,FIRSTNAME,LASTNAME,iteminsession FROM artist_song_by_user WHERE userid = 10 and sessionid = 182")
    for row in rows:
        print(f'Artist:{row.artist}#Song:{row.song}#Name:{row.firstname},{row.lastname}#{row.iteminsession}')
except Exception as e:
    print("Error in executingquery for Q2: ",e)
    

Artist:Down To The Bone#Song:Keep On Keepin' On#Name:Sylvie,Cruz#0
Artist:Three Drives#Song:Greece 2000#Name:Sylvie,Cruz#1
Artist:Sebastien Tellier#Song:Kilometer#Name:Sylvie,Cruz#2
Artist:Lonnie Gordon#Song:Catch You Baby (Steve Pitron & Max Sanna Radio Edit)#Name:Sylvie,Cruz#3


In [167]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query="CREATE TABLE IF NOT EXISTS users_by_song " \
      "(song text,userId INT,sessionId INT,iteminSession INT,firstName text,lastName text,"\
      "PRIMARY KEY(song,userId));"
try:
    session.execute(query)
except Exception as e:
    print("Error in creating table users_by_song",e)


                    

In [168]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
filerejected='rejected_event_datafile_new_q3.csv'
flag = False
with open(file, encoding = 'utf8') as f,open(filerejected,'w',encoding='utf8',newline='') as freject:
    csvreader = csv.reader(f)
    writer = csv.writer(freject, dialect='myDialect')
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users_by_song(song,userId,sessionId,iteminSession,firstName,lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        try:
            session.execute(query,(line[9],int(line[10]),int(line[8]),int(line[3]),line[1],line[4]))
        except Exception as e:
            if not flag:
                print(e)
            flag=True
            writer.writerow((line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7],line[8],line[9],line[10]))
                  #freject.write("|".join(line)+"\n")

if flag:
    print("Please check the rejected file: ",filerejected)
else:
    os.remove(filerejected)

In [170]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
#rows=session.execute("SELECT count(*) FROM users_by_song")
#for row in rows:
#    print(row)
try:
    rows=session.execute("SELECT FIRSTNAME,LASTNAME FROM users_by_song WHERE SONG='All Hands Against His Own'")
    for row in rows:
        print(f'Name:{row.firstname},{row.lastname}')
except Exception as e:
    print("Error in executingquery for Q3: ",e)
    

Name:Jacqueline,Lynch
Name:Tegan,Levine
Name:Sara,Johnson


### Drop the tables before closing out the sessions

In [171]:
## TO-DO: Drop the table before closing out the sessions

In [172]:
try:
    session.execute("DROP TABLE IF EXISTS artist_song_by_session;")
except Exception as e:
    print("Error in dropping table artist_song_by_session: ",e)

try:
    session.execute("DROP TABLE IF EXISTS artist_song_by_user;")
except Exception as e:
    print("Error in dropping table artist_song_by_user: ",e)
    
try:
    session.execute("DROP TABLE IF EXISTS users_by_song;")
except Exception as e:
    print("Error in dropping table users_by_song: ",e)    

### Close the session and cluster connection¶

In [173]:
session.shutdown()
cluster.shutdown()